In [ ]:
import os

# The current working directory needs to be in explainable_TGCNN
print(os.getcwd())
os.chdir('..\\')
print(os.getcwd())

import pandas as pd
import numpy as np
import tensorflow as tf
from src import create_fake_patients, whole_model_demographics_gradcam, graph_plot, plot_feature_value, ga, gc, utils
import statistics
import math

In [ ]:
second_TGCNN_layer = True
demo = True

include_drugs = True
max_timesteps=100

stride = 1
filter_size = 4

run_name='hip_1999_to_one_year_advance_model'
years_in_advance = "5"

if include_drugs:
    max_event_codes = 518
else:
    max_event_codes = 512
hip_or_knee = 'hip'
num_filters = 16

# fake mapping dataframe for the ReadCodes and the corresponding descriptions
read_code_map_df = pd.read_csv('fake_read_code_descriptions.csv')

model = whole_model_demographics_gradcam.TGCNN_Model(num_filters=16, num_nodes=max_event_codes, num_time_steps=max_timesteps, 
                            filter_size=filter_size, variable_gamma=True, 
                            exponential_scaling=True, dropout_rate=0.7, lstm_units=64,
                            fcl1_units=128, LSTM_ablation=False, stride=stride, activation_type='LeakyReLU', 
                            no_timestamp=False, second_TGCNN_layer=second_TGCNN_layer, num_labels=1)
model.load_weights('hip_1999_to_one_year_advance_model1_CNN_layer')

num_pats = 5
max_act_filt_num = 10
cv_patients = create_fake_patients.create_fake_patient_df(num_pats, 99, max_event_codes)

In [ ]:
def remove_nans(lst):
    cleaned_list = [x for x in lst if not math.isnan(x)]
    return cleaned_list



model2 = whole_model_demographics_gradcam.TGCNN_Model(num_filters=16, num_nodes=max_event_codes, num_time_steps=max_timesteps, 
                            filter_size=filter_size, variable_gamma=True, 
                            exponential_scaling=True, dropout_rate=0.7, lstm_units=64,
                            fcl1_units=128, LSTM_ablation=False, stride=stride, activation_type='LeakyReLU', 
                            no_timestamp=False, second_TGCNN_layer=second_TGCNN_layer, num_labels=1)
model2.load_weights(f'hip_1999_to_one_year_advance_model1_CNN_layer')


weights = model2.get_weights()

# add some random noise to each weight matrix
modified_weights = []
for w in weights:
    w_mean = np.mean(w)
    std_mean = np.std(w)
    noise = np.random.normal(w_mean, std_mean, w.shape)  # mean=0, std=0.1
    modified_w = w + noise  # Add the noise to the weights
    modified_weights.append(modified_w)

# set the modified weights back to the model
model2.set_weights(modified_weights)

model = whole_model_demographics_gradcam.TGCNN_Model(num_filters=16, num_nodes=max_event_codes, num_time_steps=max_timesteps, 
                            filter_size=filter_size, variable_gamma=True, 
                            exponential_scaling=True, dropout_rate=0.7, lstm_units=64,
                            fcl1_units=128, LSTM_ablation=False, stride=stride, activation_type='LeakyReLU', 
                            no_timestamp=False, second_TGCNN_layer=second_TGCNN_layer, num_labels=1)
model.load_weights(f'hip_1999_to_one_year_advance_model1_CNN_layer')


replacement_true_lst1, max_w_filt_lst1, filt_nums1 = ga.get_act_metric_per_feat(model, num_filters, num_pats, 
                                                                                 cv_patients, max_event_codes, hip_or_knee,
                                                                                 'max', max_timesteps)
    
mean_activation_df1 = ga.act_diff(replacement_true_lst1, max_w_filt_lst1, filt_nums1, show_plot=False)

replacement_true_lst2, max_w_filt_lst2, filt_nums2 = ga.get_act_metric_per_feat(model2, num_filters, num_pats, 
                                                                                 cv_patients, max_event_codes, hip_or_knee,
                                                                                 'max', max_timesteps)
    
mean_activation_df2 = ga.act_diff(replacement_true_lst2, max_w_filt_lst2, filt_nums2, show_plot=False)



metric_type = 'largest'

mse_diffs_mean, abs_diffs_mean = [], []
mse_diffs_median, abs_diffs_median = [], []
mse_diffs_max, abs_diffs_max = [], []
for pat_num in range(num_pats):

    if (pat_num % 50) == 0 and (pat_num !=0):
        print(f"{(((pat_num+1)/num_pats)*100):.2f}% Complete")
        mse_diffs_mean = remove_nans(mse_diffs_mean)
        abs_diffs_mean = remove_nans(abs_diffs_mean)
        mse_diffs_median = remove_nans(mse_diffs_median)
        abs_diffs_median = remove_nans(abs_diffs_median)
        mse_diffs_max = remove_nans(mse_diffs_max)
        abs_diffs_max = remove_nans(abs_diffs_max)
        print(f"MSE mean +- SD: {np.mean(mse_diffs_mean):.3f} $\pm$ {np.std(mse_diffs_mean):.3f}")
        print(f"Abs mean +- SD: {np.mean(abs_diffs_mean):.3f} $\pm$ {np.std(abs_diffs_mean):.3f}")
        
        print(f"MSE median +- SD: {np.mean(mse_diffs_median):.3f} $\pm$ {np.std(mse_diffs_median):.3f}")
        print(f"Abs median +- SD: {np.mean(abs_diffs_median):.3f} $\pm$ {np.std(abs_diffs_median):.3f}")
        
        print(f"MSE max +- SD: {np.mean(mse_diffs_max):.3f} $\pm$ {np.std(mse_diffs_max):.3f}")
        print(f"Abs max +- SD: {np.mean(abs_diffs_max):.3f} $\pm$ {np.std(abs_diffs_max):.3f}")
    
    
    
    # Generate individual data for the model
    input_3d, input_4d, demo_tensor, outcome, outcome_bin = utils.return_pat_from_df(cv_patients, max_event_codes, hip_or_knee, pat_num, max_timesteps)
    
    dense_tensor = tf.sparse.to_dense(input_3d)
    dense_tensor= tf.transpose(dense_tensor, perm=[2, 1, 0])
    dense_tensor = np.flip(dense_tensor, axis=0)

    
    
    heatmap_df = pd.DataFrame()
    # Get the entire patient's history in a DataFrame
    edges_df = graph_plot.create_edges_df_gc(dense_tensor)
    
    # Get the node positions for the graph
    pos_df = graph_plot.create_position_df_gc(edges_df)
    pos_list = graph_plot.generate_pos_sequence(pos_df['max_codes_per_visit'].max())
    pos_df = graph_plot.map_y_coord_to_node(pos_df, pos_list)
    
    
    
    for i in range(0, num_filters):
        weighted_f_map = ga.choose_feat_map(model, 'single', mean_activation_df1, i)
        timestep_ave_w_df = gc.calc_timestep_weights(1, filter_size, weighted_f_map, max_timesteps)
        read_code_pos_df = gc.map_read_code_labels(pos_df, read_code_map_df, timestep_ave_w_df)
        # remove any rows with duplicate v number in node column
        df_unique = read_code_pos_df.drop_duplicates(subset='x', keep=False)
        heatmap_df[f'col_{i}'] = df_unique['perc_timestep_infl']
    
    # add the mean, median and max map values to the heatmap too
    row_medians = heatmap_df.iloc[:, :].median(axis=1)
    heatmap_df['col_32'] = row_medians
    
    row_means = heatmap_df.iloc[:, :-1].mean(axis=1)
    heatmap_df['col_33'] = row_means
    
    weighted_f_map = ga.choose_feat_map(model, metric_type, mean_activation_df1, max_act_filt_num)
    timestep_ave_w_df = gc.calc_timestep_weights(1, filter_size, weighted_f_map, max_timesteps)
    read_code_pos_df = gc.map_read_code_labels(pos_df, read_code_map_df, timestep_ave_w_df)
    df_unique = read_code_pos_df.drop_duplicates(subset='x', keep=False)
    heatmap_df['col_34'] = df_unique['perc_timestep_infl']
    
    heatmap1_mean = heatmap_df['col_32'].to_numpy()
    heatmap1_median = heatmap_df['col_33'].to_numpy()
    heatmap1_max = heatmap_df['col_34'].to_numpy()
    
    # ------------------------------------------------------------------------------------------------------------
    
    heatmap_df2 = pd.DataFrame()
    # Get the entire patient's history in a DataFrame
    edges_df = graph_plot.create_edges_df_gc(dense_tensor)
    
    # Get the node positions for the graph
    pos_df = graph_plot.create_position_df_gc(edges_df)
    pos_list = graph_plot.generate_pos_sequence(pos_df['max_codes_per_visit'].max())
    pos_df = graph_plot.map_y_coord_to_node(pos_df, pos_list)
    
    
    
    for i in range(0, num_filters):
        weighted_f_map = ga.choose_feat_map(model2, 'single', mean_activation_df2, i)
        timestep_ave_w_df = gc.calc_timestep_weights(1, filter_size, weighted_f_map, max_timesteps)
        read_code_pos_df = gc.map_read_code_labels(pos_df, read_code_map_df, timestep_ave_w_df)
        # remove any rows with duplicate v number in node column
        df_unique = read_code_pos_df.drop_duplicates(subset='x', keep=False)
        heatmap_df2[f'col_{i}'] = df_unique['perc_timestep_infl']
    
    # add the mean, median and max map values to the heatmap too
    row_medians = heatmap_df2.iloc[:, :].median(axis=1)
    heatmap_df2['col_32'] = row_medians
    
    row_means = heatmap_df2.iloc[:, :-1].mean(axis=1)
    heatmap_df2['col_33'] = row_means
    
    weighted_f_map = ga.choose_feat_map(model2, metric_type, mean_activation_df2, max_act_filt_num)
    timestep_ave_w_df = gc.calc_timestep_weights(1, filter_size, weighted_f_map, max_timesteps)
    read_code_pos_df = gc.map_read_code_labels(pos_df, read_code_map_df, timestep_ave_w_df)
    df_unique = read_code_pos_df.drop_duplicates(subset='x', keep=False)
    heatmap_df2[f'col_{34}'] = df_unique['perc_timestep_infl']
    
    heatmap2_mean = heatmap_df2['col_32'].to_numpy()
    heatmap2_median = heatmap_df2['col_33'].to_numpy()
    heatmap2_max = heatmap_df2['col_34'].to_numpy()
    
    #mse_lst.append(mse(heatmap_array, heatmap_array2))

    ## MEAN
    mse_difference_mean = np.mean((heatmap1_mean - heatmap2_mean) ** 2)
    mse_diffs_mean.append(mse_difference_mean)
    abs_difference_mean = np.mean(np.abs(heatmap1_mean- heatmap2_mean))
    abs_diffs_mean.append(abs_difference_mean)

    ## MEDIAN
    mse_difference_median = np.mean((heatmap1_median - heatmap2_median) ** 2)
    mse_diffs_median.append(mse_difference_median)
    abs_difference_median = np.mean(np.abs(heatmap1_median- heatmap2_median))
    abs_diffs_median.append(abs_difference_median)

    ## MAX
    mse_difference_max = np.mean((heatmap1_max - heatmap2_max) ** 2)
    mse_diffs_max.append(mse_difference_max)
    abs_difference_max = np.mean(np.abs(heatmap1_max- heatmap2_max))
    abs_diffs_max.append(abs_difference_max)
    

mse_diffs_mean = remove_nans(mse_diffs_mean)
abs_diffs_mean = remove_nans(abs_diffs_mean)
mse_diffs_median = remove_nans(mse_diffs_median)
abs_diffs_median = remove_nans(abs_diffs_median)
mse_diffs_max = remove_nans(mse_diffs_max)
abs_diffs_max = remove_nans(abs_diffs_max)
print(f"MSE mean +- SD: {np.mean(mse_diffs_mean):.3f} $\pm$ {np.std(mse_diffs_mean):.3f}")
print(f"Abs mean +- SD: {np.mean(abs_diffs_mean):.3f} $\pm$ {np.std(abs_diffs_mean):.3f}")

print(f"MSE median +- SD: {np.mean(mse_diffs_median):.3f} $\pm$ {np.std(mse_diffs_median):.3f}")
print(f"Abs median +- SD: {np.mean(abs_diffs_median):.3f} $\pm$ {np.std(abs_diffs_median):.3f}")

print(f"MSE max +- SD: {np.mean(mse_diffs_max):.3f} $\pm$ {np.std(mse_diffs_max):.3f}")
print(f"Abs max +- SD: {np.mean(abs_diffs_max):.3f} $\pm$ {np.std(abs_diffs_max):.3f}")